# Redes Neurais Artificias

As Redes Neurais Artificiais simulam a maneira de transmissão e aprendizagem de informação dos neurônios presentes no cérebro humano. RNA’s são definidas como uma técnica de otimização onde se busca por uma hipótese de uma função recorrendo a otimização da mesma, ou seja, minimizar (ou otimizar) uma função.

Existem diversos modelos de ANN: Perceptron, Auto Encoder, Convolution, Kohonen são alguns exemplos. Neste trabalho utilizamos a Rede Neural Perceptron Multicamadas (MLP). A rede MLP é um modelo composto por múltiplas camadas de neurônios artificiais totalmente conectadas, denominadas: Camada de Entrada, Camada Intermediária e Camada de Saída. Um exemplo de sua arquitetura pode ser visualizada na figura a seguir.

![MLP](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQAGlDDqdEvLbBHwJw9qpYzXHMJFhO7wB1EU0rL5Bn-bqUVIl5cyw)


Para que a MLP consiga prever com exatidão a hipótese é necessário realizar seu treinamento. Por se tratar de um algoritmo de aprendizado supervisionado, devemos apresentar o conjunto $(x^{(1)}, y^{(1)}), (x^{(2)}, y^{(2)}), ..., (x^{(m)}, y^{(m)})$. A seguir são apresentados os resultados dos algoritmos de aprendizagem supervisionado. Foram realizados dois teste. No primeiro os algoritmos receberam como entrada NDVI, Diferença do NVDI, NBRL, Diferença do NBRL. O conjunto de entrada neste caso ficou definido como $x^{(i)} \in \mathbb{R}^{4} y^{(i)} \in {0,1} $. No segundo teste os algoritmos receberam como entrada os atributos dos índices espectrais NDVI, Diferença do NVDI, NBRL, Diferença do NBRL, e as medianas das bandas $[2,5]$. O conjunto de entrada neste caso ficou definido como $x^{(i)} \in \mathbb{R}^{8} y^{(i)} \in {0,1} $.

### Código MLP usando o Sckit Learn

Realizando os **import** das bibliotecas necessárias para realizar o treinamento.

In [3]:
from sklearn.model_selection import train_test_split
import numpy as np
import gdal, gdalconst
import geopandas as gpd
from sklearn import model_selection
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt

Criando um **`GeoDataFrame`** com o shapely de queimadas.

In [6]:
df = gpd.read_file("DS/221_067_2017.shp")

Chamando o método **`head()`** do **Pandas** para visualizar os atributos do objeto.

In [7]:
df.head()

,id,cod_sat,cena_id,nome_arq,orb_pto,area_ha,perim,n_arq_ant,ndvi,nbrl,...,medianb4,medianb5,medianb6,medianb7,verifica,proc_id,focos,data_atual,data_anter,geometry
0,23946449,8,LC82210672017125LGN00,LC82210672017125LGN00.tar.gz,221_067,12.968645,2639,LC82210672017109LGN00.tar.bz,0.211523,-0.044128,...,0.144337,0.227026,0.304834,0.245830,1,5886,2,2017-05-05,2017-04-19 00:00:00,"POLYGON ((-46.9715795250224 -10.3363584942161,..."
1,23946450,8,LC82210672017125LGN00,LC82210672017125LGN00.tar.gz,221_067,1.263320,600,LC82210672017109LGN00.tar.bz,0.042114,-0.027508,...,0.040925,0.044027,0.049672,0.046862,1,5886,0,2017-05-05,2017-04-19 00:00:00,"POLYGON ((-46.358025045795 -10.3373571795547, ..."
2,23946451,8,LC82210672017125LGN00,LC82210672017125LGN00.tar.gz,221_067,7.568930,2340,LC82210672017109LGN00.tar.bz,0.306289,-0.003284,...,0.083032,0.151609,0.205350,0.169358,1,5886,1,2017-05-05,2017-04-19 00:00:00,"POLYGON ((-46.9860921817999 -10.3359982294198,..."
3,23946452,8,LC82210672017125LGN00,LC82210672017125LGN00.tar.gz,221_067,1.804195,660,LC82210672017109LGN00.tar.bz,0.256839,-0.017937,...,0.143473,0.242181,0.360290,0.251297,0,5886,0,2017-05-05,2017-04-19 00:00:00,"POLYGON ((-46.0128759745859 -10.3521908768478,..."
4,23946453,8,LC82210672017125LGN00,LC82210672017125LGN00.tar.gz,221_067,2.886433,1140,LC82210672017109LGN00.tar.bz,0.240240,0.000871,...,0.116126,0.195281,0.244584,0.189877,1,5886,2,2017-05-05,2017-04-19 00:00:00,"POLYGON ((-46.3574643464459 -10.3343756813685,..."


Será realizado dois testes com a MLP. No primeiro, o algoritmo será treinado somente com os índices espectrais **` NDVI`, `DIF_NDVI`, `NBRL`, `DIF_NBRL`** e no segundo teste além dos índices espectrais, será usado as medianas das bandas [2 à 5] do arquivo shapely para o treinamento.


#### - Teste 1

No próximo passo, será criado duas váriaveis com dados a ser passado ao algoritmo de treinamento. 

- `x` - Essa variável contém como dados de entrada os índices espectrais **`ndvi`**, **`nbrl`**, e suas diferenças o **`dif_ndvi`** e o **`dif_dnbrl`**. 

- `y` - Essa variável contém os dados da coluna **`verifica`**.


In [8]:
x = np.array(df[['ndvi','nbrl','dif_ndvi','dif_dnbrl']])
y = np.array(df[['verifica']])

Com os dados separado, será definido o modelo da **Rede Neural Artificial Multilayer**. Os parâmetros foram definidos de forma empirica.

In [9]:
clf = MLPClassifier(activation='identity', solver='adam', verbose=True, learning_rate_init=0.00001,
                    alpha=1e-5,hidden_layer_sizes=(80,2),  momentum=0.7, random_state=1, max_iter=1200)

Uma das metologias de avaliação consiste em dividir os dados em dois conjuntos: teste e treinamento, calculando a pontuação 7 vezes consecutivas (com diferentes divisões de cada vez).

Assim é definido no atributo de entrada **random_state** da função **KFold** o valor 7.

In [16]:
kfold = model_selection.KFold(n_splits=10, random_state=7)


Para avaliar a performance dos algoritmos foi utilizada a função Cross-validation da biblioteca scikit-learn. A avaliação dos dados não deve ser feita utilizando o conjunto de treinamento.

Dito isso, agora é realizado o treinamento:

In [17]:
y = y.ravel()
cv_results = model_selection.cross_val_score(clf, x, y, cv=kfold, scoring='accuracy')

Iteration 1, loss = 0.73628137
Iteration 2, loss = 0.73219083
Iteration 3, loss = 0.72831176
Iteration 4, loss = 0.72461181
Iteration 5, loss = 0.72109237
Iteration 6, loss = 0.71773563
Iteration 7, loss = 0.71456291
Iteration 8, loss = 0.71156365
Iteration 9, loss = 0.70871941
Iteration 10, loss = 0.70601857
Iteration 11, loss = 0.70349401
Iteration 12, loss = 0.70110945
Iteration 13, loss = 0.69887461
Iteration 14, loss = 0.69676361
Iteration 15, loss = 0.69477623
Iteration 16, loss = 0.69292899
Iteration 17, loss = 0.69117934
Iteration 18, loss = 0.68954142
Iteration 19, loss = 0.68800179
Iteration 20, loss = 0.68656979
Iteration 21, loss = 0.68521460
Iteration 22, loss = 0.68394877
Iteration 23, loss = 0.68276039
Iteration 24, loss = 0.68163338
Iteration 25, loss = 0.68057293
Iteration 26, loss = 0.67955545
Iteration 27, loss = 0.67859946
Iteration 28, loss = 0.67768865
Iteration 29, loss = 0.67681056
Iteration 30, loss = 0.67597170
Iteration 31, loss = 0.67516794
Iteration 32, los

Iteration 253, loss = 0.52494155
Iteration 254, loss = 0.52450168
Iteration 255, loss = 0.52406489
Iteration 256, loss = 0.52363496
Iteration 257, loss = 0.52320376
Iteration 258, loss = 0.52277501
Iteration 259, loss = 0.52235900
Iteration 260, loss = 0.52193361
Iteration 261, loss = 0.52151460
Iteration 262, loss = 0.52110239
Iteration 263, loss = 0.52068788
Iteration 264, loss = 0.52027936
Iteration 265, loss = 0.51987975
Iteration 266, loss = 0.51947562
Iteration 267, loss = 0.51908147
Iteration 268, loss = 0.51868574
Iteration 269, loss = 0.51829570
Iteration 270, loss = 0.51790209
Iteration 271, loss = 0.51751952
Iteration 272, loss = 0.51714860
Iteration 273, loss = 0.51676331
Iteration 274, loss = 0.51638674
Iteration 275, loss = 0.51601363
Iteration 276, loss = 0.51564672
Iteration 277, loss = 0.51527569
Iteration 278, loss = 0.51491768
Iteration 279, loss = 0.51455985
Iteration 280, loss = 0.51419391
Iteration 281, loss = 0.51384367
Iteration 282, loss = 0.51349712
Iteration 

Iteration 113, loss = 0.62411261
Iteration 114, loss = 0.62343344
Iteration 115, loss = 0.62276051
Iteration 116, loss = 0.62208374
Iteration 117, loss = 0.62140005
Iteration 118, loss = 0.62072590
Iteration 119, loss = 0.62004279
Iteration 120, loss = 0.61936471
Iteration 121, loss = 0.61869073
Iteration 122, loss = 0.61801279
Iteration 123, loss = 0.61733889
Iteration 124, loss = 0.61666368
Iteration 125, loss = 0.61598988
Iteration 126, loss = 0.61532084
Iteration 127, loss = 0.61464553
Iteration 128, loss = 0.61397443
Iteration 129, loss = 0.61330413
Iteration 130, loss = 0.61264219
Iteration 131, loss = 0.61196752
Iteration 132, loss = 0.61130523
Iteration 133, loss = 0.61064074
Iteration 134, loss = 0.60997687
Iteration 135, loss = 0.60931734
Iteration 136, loss = 0.60865201
Iteration 137, loss = 0.60798960
Iteration 138, loss = 0.60732762
Iteration 139, loss = 0.60667294
Iteration 140, loss = 0.60601473
Iteration 141, loss = 0.60536339
Iteration 142, loss = 0.60470377
Iteration 

Iteration 366, loss = 0.51821095
Iteration 367, loss = 0.51808257
Iteration 368, loss = 0.51795687
Iteration 369, loss = 0.51783540
Iteration 370, loss = 0.51771578
Iteration 371, loss = 0.51760610
Iteration 372, loss = 0.51747826
Iteration 373, loss = 0.51735872
Iteration 374, loss = 0.51724474
Iteration 375, loss = 0.51713087
Iteration 376, loss = 0.51702496
Iteration 377, loss = 0.51690490
Iteration 378, loss = 0.51679189
Iteration 379, loss = 0.51668677
Iteration 380, loss = 0.51658411
Iteration 381, loss = 0.51647567
Iteration 382, loss = 0.51637414
Iteration 383, loss = 0.51626813
Iteration 384, loss = 0.51616506
Iteration 385, loss = 0.51607063
Iteration 386, loss = 0.51596511
Iteration 387, loss = 0.51587970
Iteration 388, loss = 0.51578103
Iteration 389, loss = 0.51568577
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 1, loss = 0.74969870
Iteration 2, loss = 0.74493354
Iteration 3, loss = 0.74037079
Iteration 4, loss = 0.73

Iteration 231, loss = 0.52238319
Iteration 232, loss = 0.52188021
Iteration 233, loss = 0.52138438
Iteration 234, loss = 0.52089331
Iteration 235, loss = 0.52039828
Iteration 236, loss = 0.51991639
Iteration 237, loss = 0.51942955
Iteration 238, loss = 0.51895508
Iteration 239, loss = 0.51847432
Iteration 240, loss = 0.51801107
Iteration 241, loss = 0.51754582
Iteration 242, loss = 0.51708384
Iteration 243, loss = 0.51662733
Iteration 244, loss = 0.51617422
Iteration 245, loss = 0.51572674
Iteration 246, loss = 0.51527982
Iteration 247, loss = 0.51483940
Iteration 248, loss = 0.51440305
Iteration 249, loss = 0.51396655
Iteration 250, loss = 0.51352910
Iteration 251, loss = 0.51310256
Iteration 252, loss = 0.51267641
Iteration 253, loss = 0.51226232
Iteration 254, loss = 0.51184192
Iteration 255, loss = 0.51143147
Iteration 256, loss = 0.51102236
Iteration 257, loss = 0.51061582
Iteration 258, loss = 0.51021061
Iteration 259, loss = 0.50981258
Iteration 260, loss = 0.50941513
Iteration 

...

Iteration 80, loss = 0.62823774
Iteration 81, loss = 0.62735997
Iteration 82, loss = 0.62648078
Iteration 83, loss = 0.62560410
Iteration 84, loss = 0.62473188
Iteration 85, loss = 0.62384346
Iteration 86, loss = 0.62296128
Iteration 87, loss = 0.62207610
Iteration 88, loss = 0.62119187
Iteration 89, loss = 0.62030181
Iteration 90, loss = 0.61941452
Iteration 91, loss = 0.61852657
Iteration 92, loss = 0.61764199
Iteration 93, loss = 0.61674513
Iteration 94, loss = 0.61585410
Iteration 95, loss = 0.61496166
Iteration 96, loss = 0.61407282
Iteration 97, loss = 0.61317920
Iteration 98, loss = 0.61228656
Iteration 99, loss = 0.61139620
Iteration 100, loss = 0.61050898
Iteration 101, loss = 0.60962378
Iteration 102, loss = 0.60873107
Iteration 103, loss = 0.60784186
Iteration 104, loss = 0.60695045
Iteration 105, loss = 0.60606127
Iteration 106, loss = 0.60517064
Iteration 107, loss = 0.60428141
Iteration 108, loss = 0.60339187
Iteration 109, loss = 0.60249919
Iteration 110, loss = 0.601606

Iteration 332, loss = 0.46426499
Iteration 333, loss = 0.46400900
Iteration 334, loss = 0.46375132
Iteration 335, loss = 0.46349704
Iteration 336, loss = 0.46324850
Iteration 337, loss = 0.46299676
Iteration 338, loss = 0.46275822
Iteration 339, loss = 0.46251400
Iteration 340, loss = 0.46227670
Iteration 341, loss = 0.46203944
Iteration 342, loss = 0.46180371
Iteration 343, loss = 0.46157370
Iteration 344, loss = 0.46134634
Iteration 345, loss = 0.46111959
Iteration 346, loss = 0.46090204
Iteration 347, loss = 0.46067359
Iteration 348, loss = 0.46045355
Iteration 349, loss = 0.46023905
Iteration 350, loss = 0.46002870
Iteration 351, loss = 0.45981409
Iteration 352, loss = 0.45960930
Iteration 353, loss = 0.45939698
Iteration 354, loss = 0.45918692
Iteration 355, loss = 0.45898485
Iteration 356, loss = 0.45878647
Iteration 357, loss = 0.45859389
Iteration 358, loss = 0.45839731
Iteration 359, loss = 0.45819469
Iteration 360, loss = 0.45799877
Iteration 361, loss = 0.45781405
Iteration 

Iteration 166, loss = 0.56276609
Iteration 167, loss = 0.56200637
Iteration 168, loss = 0.56124328
Iteration 169, loss = 0.56048597
Iteration 170, loss = 0.55973906
Iteration 171, loss = 0.55898900
Iteration 172, loss = 0.55824732
Iteration 173, loss = 0.55749984
Iteration 174, loss = 0.55676112
Iteration 175, loss = 0.55602492
Iteration 176, loss = 0.55529423
Iteration 177, loss = 0.55456092
Iteration 178, loss = 0.55383496
Iteration 179, loss = 0.55310929
Iteration 180, loss = 0.55239136
Iteration 181, loss = 0.55166383
Iteration 182, loss = 0.55094726
Iteration 183, loss = 0.55023846
Iteration 184, loss = 0.54953554
Iteration 185, loss = 0.54881658
Iteration 186, loss = 0.54812000
Iteration 187, loss = 0.54741582
Iteration 188, loss = 0.54672391
Iteration 189, loss = 0.54603392
Iteration 190, loss = 0.54534117
Iteration 191, loss = 0.54465735
Iteration 192, loss = 0.54398099
Iteration 193, loss = 0.54329815
Iteration 194, loss = 0.54262629
Iteration 195, loss = 0.54195126
Iteration 

Iteration 14, loss = 0.69047769
Iteration 15, loss = 0.68884527
Iteration 16, loss = 0.68731655
Iteration 17, loss = 0.68585864
Iteration 18, loss = 0.68449821
Iteration 19, loss = 0.68321365
Iteration 20, loss = 0.68199553
Iteration 21, loss = 0.68084805
Iteration 22, loss = 0.67974751
Iteration 23, loss = 0.67869779
Iteration 24, loss = 0.67769363
Iteration 25, loss = 0.67673491
Iteration 26, loss = 0.67579458
Iteration 27, loss = 0.67488645
Iteration 28, loss = 0.67401507
Iteration 29, loss = 0.67316103
Iteration 30, loss = 0.67232157
Iteration 31, loss = 0.67149912
Iteration 32, loss = 0.67068869
Iteration 33, loss = 0.66988775
Iteration 34, loss = 0.66908776
Iteration 35, loss = 0.66829917
Iteration 36, loss = 0.66751986
Iteration 37, loss = 0.66673546
Iteration 38, loss = 0.66595463
Iteration 39, loss = 0.66517657
Iteration 40, loss = 0.66440127
Iteration 41, loss = 0.66361930
Iteration 42, loss = 0.66283809
Iteration 43, loss = 0.66205591
Iteration 44, loss = 0.66127271
Iteratio

Iteration 271, loss = 0.50951568
Iteration 272, loss = 0.50920462
Iteration 273, loss = 0.50889470
Iteration 274, loss = 0.50858046
Iteration 275, loss = 0.50827558
Iteration 276, loss = 0.50797544
Iteration 277, loss = 0.50767660
Iteration 278, loss = 0.50737999
Iteration 279, loss = 0.50708932
Iteration 280, loss = 0.50679348
Iteration 281, loss = 0.50650590
Iteration 282, loss = 0.50622797
Iteration 283, loss = 0.50594023
Iteration 284, loss = 0.50566930
Iteration 285, loss = 0.50538514
Iteration 286, loss = 0.50511706
Iteration 287, loss = 0.50483820
Iteration 288, loss = 0.50457164
Iteration 289, loss = 0.50430577
Iteration 290, loss = 0.50404191
Iteration 291, loss = 0.50377950
Iteration 292, loss = 0.50352075
Iteration 293, loss = 0.50326172
Iteration 294, loss = 0.50300901
Iteration 295, loss = 0.50275952
Iteration 296, loss = 0.50251055
Iteration 297, loss = 0.50226817
Iteration 298, loss = 0.50202156
Iteration 299, loss = 0.50178603
Iteration 300, loss = 0.50154822
Iteration 

O resultado da execução desse algoritmo, logo abaixo, será usado na seção [3 Resultados e Conclusões](https://github.com/GSansigolo/CAP-240-394/blob/master/src/conclusao/conclusao.ipynb) para uma comparação gráfica dos algoritmos de aprendizado de máquina. 

In [19]:
print(cv_results)

[0.74792597 0.9023612  0.720485   0.71410338 0.73261008 0.7626037
 0.76451819 0.57115507 0.62667518 0.69687301]


#### - Teste 2

No próximo passo, será criado duas váriaveis com dados a ser passado ao algoritmo de treinamento. 

- `x` - Essa variável contém como dados de entrada os índices espectrais **`ndvi`**, **`nbrl`**, suas diferenças o **`dif_ndvi`** e o **`dif_dnbrl`**, e as **medianas das bandas [2,5]**. 

- `y` - Essa variável contém os dados da coluna **`verifica`**.


In [21]:
x = np.array(df[['ndvi','nbrl','dif_ndvi','dif_dnbrl','medianb2','medianb3','medianb4','medianb5']])
y = np.array(df[['verifica']])

Será usado as mesmas configurações da rede, realizadas no **Teste 1**.

In [22]:
y = y.ravel()
cv_results = model_selection.cross_val_score(clf, x, y, cv=kfold, scoring=scoring)

Iteration 1, loss = 0.73338756
Iteration 2, loss = 0.72909954
Iteration 3, loss = 0.72502381
Iteration 4, loss = 0.72112837
Iteration 5, loss = 0.71741628
Iteration 6, loss = 0.71387032
Iteration 7, loss = 0.71051368
Iteration 8, loss = 0.70733442
Iteration 9, loss = 0.70431646
Iteration 10, loss = 0.70144719
Iteration 11, loss = 0.69876117
Iteration 12, loss = 0.69622205
Iteration 13, loss = 0.69384170
Iteration 14, loss = 0.69159176
Iteration 15, loss = 0.68947375
Iteration 16, loss = 0.68750556
Iteration 17, loss = 0.68564394
Iteration 18, loss = 0.68390339
Iteration 19, loss = 0.68227069
Iteration 20, loss = 0.68075519
Iteration 21, loss = 0.67932506
Iteration 22, loss = 0.67799447
Iteration 23, loss = 0.67675085
Iteration 24, loss = 0.67557754
Iteration 25, loss = 0.67447964
Iteration 26, loss = 0.67343312
Iteration 27, loss = 0.67245627
Iteration 28, loss = 0.67153246
Iteration 29, loss = 0.67064781
Iteration 30, loss = 0.66981018
Iteration 31, loss = 0.66901328
Iteration 32, los

Iteration 256, loss = 0.52364907
Iteration 257, loss = 0.52322491
Iteration 258, loss = 0.52280271
Iteration 259, loss = 0.52239321
Iteration 260, loss = 0.52197383
Iteration 261, loss = 0.52156048
Iteration 262, loss = 0.52115411
Iteration 263, loss = 0.52074453
Iteration 264, loss = 0.52034110
Iteration 265, loss = 0.51994641
Iteration 266, loss = 0.51954688
Iteration 267, loss = 0.51915686
Iteration 268, loss = 0.51876520
Iteration 269, loss = 0.51837896
Iteration 270, loss = 0.51798852
Iteration 271, loss = 0.51760888
Iteration 272, loss = 0.51724083
Iteration 273, loss = 0.51685823
Iteration 274, loss = 0.51648390
Iteration 275, loss = 0.51611295
Iteration 276, loss = 0.51574784
Iteration 277, loss = 0.51537834
Iteration 278, loss = 0.51502192
Iteration 279, loss = 0.51466546
Iteration 280, loss = 0.51430002
Iteration 281, loss = 0.51395076
Iteration 282, loss = 0.51360461
Iteration 283, loss = 0.51324991
Iteration 284, loss = 0.51291453
Iteration 285, loss = 0.51256665
Iteration 

Iteration 105, loss = 0.62588602
Iteration 106, loss = 0.62523966
Iteration 107, loss = 0.62459470
Iteration 108, loss = 0.62395323
Iteration 109, loss = 0.62330529
Iteration 110, loss = 0.62265883
Iteration 111, loss = 0.62201176
Iteration 112, loss = 0.62136539
Iteration 113, loss = 0.62071554
Iteration 114, loss = 0.62006563
Iteration 115, loss = 0.61942186
Iteration 116, loss = 0.61877461
Iteration 117, loss = 0.61811976
Iteration 118, loss = 0.61747470
Iteration 119, loss = 0.61682079
Iteration 120, loss = 0.61617158
Iteration 121, loss = 0.61552653
Iteration 122, loss = 0.61487765
Iteration 123, loss = 0.61423284
Iteration 124, loss = 0.61358635
Iteration 125, loss = 0.61294143
Iteration 126, loss = 0.61230129
Iteration 127, loss = 0.61165444
Iteration 128, loss = 0.61101199
Iteration 129, loss = 0.61037007
Iteration 130, loss = 0.60973655
Iteration 131, loss = 0.60908939
Iteration 132, loss = 0.60845521
Iteration 133, loss = 0.60781898
Iteration 134, loss = 0.60718280
Iteration 

Iteration 361, loss = 0.51810301
Iteration 362, loss = 0.51796054
Iteration 363, loss = 0.51782104
Iteration 364, loss = 0.51767932
Iteration 365, loss = 0.51754652
Iteration 366, loss = 0.51741248
Iteration 367, loss = 0.51727647
Iteration 368, loss = 0.51714312
Iteration 369, loss = 0.51701398
Iteration 370, loss = 0.51688656
Iteration 371, loss = 0.51676924
Iteration 372, loss = 0.51663361
Iteration 373, loss = 0.51650634
Iteration 374, loss = 0.51638481
Iteration 375, loss = 0.51626340
Iteration 376, loss = 0.51614998
Iteration 377, loss = 0.51602197
Iteration 378, loss = 0.51590120
Iteration 379, loss = 0.51578832
Iteration 380, loss = 0.51567825
Iteration 381, loss = 0.51556206
Iteration 382, loss = 0.51545283
Iteration 383, loss = 0.51533933
Iteration 384, loss = 0.51522838
Iteration 385, loss = 0.51512644
Iteration 386, loss = 0.51501330
Iteration 387, loss = 0.51492010
Iteration 388, loss = 0.51481413
Iteration 389, loss = 0.51471131
Iteration 390, loss = 0.51461087
Iteration 

Iteration 221, loss = 0.52547520
Iteration 222, loss = 0.52495516
Iteration 223, loss = 0.52443487
Iteration 224, loss = 0.52392120
Iteration 225, loss = 0.52341056
Iteration 226, loss = 0.52290760
Iteration 227, loss = 0.52241429
Iteration 228, loss = 0.52191728
Iteration 229, loss = 0.52142572
Iteration 230, loss = 0.52093713
Iteration 231, loss = 0.52045056
Iteration 232, loss = 0.51996925
Iteration 233, loss = 0.51949530
Iteration 234, loss = 0.51902562
Iteration 235, loss = 0.51855177
Iteration 236, loss = 0.51809117
Iteration 237, loss = 0.51762510
Iteration 238, loss = 0.51717115
Iteration 239, loss = 0.51670979
Iteration 240, loss = 0.51626625
Iteration 241, loss = 0.51582010
Iteration 242, loss = 0.51537696
Iteration 243, loss = 0.51493876
Iteration 244, loss = 0.51450382
Iteration 245, loss = 0.51407452
Iteration 246, loss = 0.51364532
Iteration 247, loss = 0.51322294
Iteration 248, loss = 0.51280348
Iteration 249, loss = 0.51238475
Iteration 250, loss = 0.51196338
Iteration 

...

Iteration 35, loss = 0.65865603
Iteration 36, loss = 0.65772561
Iteration 37, loss = 0.65682472
Iteration 38, loss = 0.65596419
Iteration 39, loss = 0.65512805
Iteration 40, loss = 0.65432143
Iteration 41, loss = 0.65352863
Iteration 42, loss = 0.65275208
Iteration 43, loss = 0.65198973
Iteration 44, loss = 0.65123931
Iteration 45, loss = 0.65049605
Iteration 46, loss = 0.64975788
Iteration 47, loss = 0.64902612
Iteration 48, loss = 0.64830183
Iteration 49, loss = 0.64757438
Iteration 50, loss = 0.64685266
Iteration 51, loss = 0.64612714
Iteration 52, loss = 0.64540145
Iteration 53, loss = 0.64467146
Iteration 54, loss = 0.64394294
Iteration 55, loss = 0.64320876
Iteration 56, loss = 0.64247140
Iteration 57, loss = 0.64173003
Iteration 58, loss = 0.64098724
Iteration 59, loss = 0.64023672
Iteration 60, loss = 0.63948537
Iteration 61, loss = 0.63873008
Iteration 62, loss = 0.63796703
Iteration 63, loss = 0.63720170
Iteration 64, loss = 0.63643144
Iteration 65, loss = 0.63565950
Iteratio

Iteration 289, loss = 0.47883554
Iteration 290, loss = 0.47845133
Iteration 291, loss = 0.47806007
Iteration 292, loss = 0.47768318
Iteration 293, loss = 0.47730087
Iteration 294, loss = 0.47692546
Iteration 295, loss = 0.47655391
Iteration 296, loss = 0.47618703
Iteration 297, loss = 0.47582677
Iteration 298, loss = 0.47545574
Iteration 299, loss = 0.47510236
Iteration 300, loss = 0.47474542
Iteration 301, loss = 0.47438943
Iteration 302, loss = 0.47404006
Iteration 303, loss = 0.47369176
Iteration 304, loss = 0.47334547
Iteration 305, loss = 0.47300944
Iteration 306, loss = 0.47266553
Iteration 307, loss = 0.47233076
Iteration 308, loss = 0.47199634
Iteration 309, loss = 0.47166440
Iteration 310, loss = 0.47133381
Iteration 311, loss = 0.47101355
Iteration 312, loss = 0.47068985
Iteration 313, loss = 0.47037712
Iteration 314, loss = 0.47005634
Iteration 315, loss = 0.46974395
Iteration 316, loss = 0.46943426
Iteration 317, loss = 0.46912700
Iteration 318, loss = 0.46882255
Iteration 

Iteration 112, loss = 0.60495736
Iteration 113, loss = 0.60415090
Iteration 114, loss = 0.60334634
Iteration 115, loss = 0.60254152
Iteration 116, loss = 0.60173643
Iteration 117, loss = 0.60093668
Iteration 118, loss = 0.60013124
Iteration 119, loss = 0.59932614
Iteration 120, loss = 0.59851603
Iteration 121, loss = 0.59771515
Iteration 122, loss = 0.59691342
Iteration 123, loss = 0.59610619
Iteration 124, loss = 0.59530000
Iteration 125, loss = 0.59449205
Iteration 126, loss = 0.59368725
Iteration 127, loss = 0.59288183
Iteration 128, loss = 0.59207669
Iteration 129, loss = 0.59127071
Iteration 130, loss = 0.59046587
Iteration 131, loss = 0.58966407
Iteration 132, loss = 0.58885599
Iteration 133, loss = 0.58805511
Iteration 134, loss = 0.58725507
Iteration 135, loss = 0.58645475
Iteration 136, loss = 0.58565826
Iteration 137, loss = 0.58485896
Iteration 138, loss = 0.58406330
Iteration 139, loss = 0.58326467
Iteration 140, loss = 0.58247081
Iteration 141, loss = 0.58168372
Iteration 

Iteration 363, loss = 0.47629975
Iteration 364, loss = 0.47613785
Iteration 365, loss = 0.47597647
Iteration 366, loss = 0.47581175
Iteration 367, loss = 0.47565515
Iteration 368, loss = 0.47550187
Iteration 369, loss = 0.47534654
Iteration 370, loss = 0.47519236
Iteration 371, loss = 0.47504154
Iteration 372, loss = 0.47489157
Iteration 373, loss = 0.47474744
Iteration 374, loss = 0.47459609
Iteration 375, loss = 0.47445391
Iteration 376, loss = 0.47430937
Iteration 377, loss = 0.47416827
Iteration 378, loss = 0.47403189
Iteration 379, loss = 0.47389699
Iteration 380, loss = 0.47375545
Iteration 381, loss = 0.47363126
Iteration 382, loss = 0.47349983
Iteration 383, loss = 0.47336114
Iteration 384, loss = 0.47323319
Iteration 385, loss = 0.47311105
Iteration 386, loss = 0.47298645
Iteration 387, loss = 0.47286228
Iteration 388, loss = 0.47274313
Iteration 389, loss = 0.47262863
Iteration 390, loss = 0.47250447
Iteration 391, loss = 0.47238988
Iteration 392, loss = 0.47228490
Iteration 

Iteration 208, loss = 0.53662621
Iteration 209, loss = 0.53608957
Iteration 210, loss = 0.53556371
Iteration 211, loss = 0.53503801
Iteration 212, loss = 0.53452085
Iteration 213, loss = 0.53401267
Iteration 214, loss = 0.53349281
Iteration 215, loss = 0.53297752
Iteration 216, loss = 0.53247734
Iteration 217, loss = 0.53197365
Iteration 218, loss = 0.53147592
Iteration 219, loss = 0.53098384
Iteration 220, loss = 0.53048656
Iteration 221, loss = 0.52999854
Iteration 222, loss = 0.52951471
Iteration 223, loss = 0.52903258
Iteration 224, loss = 0.52855448
Iteration 225, loss = 0.52808708
Iteration 226, loss = 0.52760980
Iteration 227, loss = 0.52714443
Iteration 228, loss = 0.52667995
Iteration 229, loss = 0.52622450
Iteration 230, loss = 0.52576664
Iteration 231, loss = 0.52532057
Iteration 232, loss = 0.52486635
Iteration 233, loss = 0.52442063
Iteration 234, loss = 0.52398098
Iteration 235, loss = 0.52353706
Iteration 236, loss = 0.52310012
Iteration 237, loss = 0.52267633
Iteration 

Como foi dito logo acima, o resultado da execução desse algoritmo será usado na seção [3 Resultados e Conclusões](https://github.com/GSansigolo/CAP-240-394/blob/master/src/conclusao/conclusao.ipynb) para uma comparação gráfica dos algoritmos de aprendizado de máquina. 

In [23]:
print(cv_results)

[0.74473516 0.90044671 0.70644544 0.70835992 0.72941927 0.76579451
 0.76643267 0.56987875 0.62986599 0.70580728]
